### MOTIF-BASED EXPLAINER

This notebook represents the analysis of the previously built datasets to explain changes in inclusion levels of Knockdown responsive exons just based on motif ocurrences. 
Each RBP represents a dataset. Ideally, the motifs that would explain given dPSI values (or exon strength) would highlight the RBPs that were knockdown in the given experiment. 

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
from explainer.datasets.global_explain import GlobalDataset
from gtfhandle.utils import file_to_bed_df

join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.
join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


Let's import the spliceAI predictions for all the union of the different exons present in all the datasets

In [2]:
# SpliceAI predictions for all exon coordinates
SPLICEAI_PREDS_PATH = "/Users/pbarbosa/Desktop/NFS_Pedro/phd/rbp_data/HepG2/tabular_dataset_generation/FINAL_SPLICEAI_PREDS.tsv.gz"
#SPLICEAI_PREDS_PATH = "/Users/pbarbosa/git_repos/interpret_splicing/notebooks/1_BUILD_DATASETS/FINAL_SPLICEAI_PREDS.tsv.gz"
SPLICEAI_PREDS = pd.read_csv(SPLICEAI_PREDS_PATH, sep="\t", low_memory=False)

# Final tabular output 
#FINAL_TABULAR = pd.read_csv("/Users/pbarbosa/Desktop/NFS_Pedro/phd/rbp_data/HepG2/tabular_dataset_generation/FINAL_READY_DATASET.tsv.gz", sep="\t", low_memory=False)

# GTF CACHE
GTF_CACHE = "/Users/pbarbosa/Desktop/NFS_Pedro/phd/rbp_data/HepG2/cache_gtf/"

# FASTA 
FASTA = "/Users/pbarbosa/MEOCloud/genome_utilities/hg38/GRCh38.primary_assembly.genome.fa"

In [3]:
cols_to_drop = ['ref_donor_cassette', 'ref_acceptor_cassette',
                'ref_acceptor_upstream', 'ref_donor_downstream',
                'ref_donor_upstream', 'ref_acceptor_downstream']

def generateDatasets(df: pd.DataFrame, 
                     analysis_name: str,
                     use_motif_counts: bool,
                     use_motif_locations: bool,
                     use_motif_distances: bool, 
                     **kwargs):
    """
    Generates tabular datasets
    """
    #print("Processing {}".format(name))
    os.makedirs('tabular_datasets', exist_ok=True)
    
    map_d = {'occurrences_alone': 'S1', 'occurrences_based_on_location': 'S2', 'occurrences_based_on_location_and_distances': 'S3'}
    analysis_name = map_d.get(analysis_name)
   
    out = []
    for exon_group, data in df.groupby('exon_group'):
        
        rbp = df.name
        outdir = os.path.join(os.path.dirname(SPLICEAI_PREDS_PATH), rbp + "_" + exon_group)
        kwargs['outbasename'] = rbp + "_" + exon_group
        
        dt = GlobalDataset(data,
                    outdir=outdir,
                    normalize_by_length=True,
                    use_motif_counts=use_motif_counts,
                    use_submotif_counts=False,
                    use_motif_locations=use_motif_locations,
                    use_motif_distances=use_motif_distances,
                    **kwargs)
        
        dt.data = dt.data.drop(columns=cols_to_drop + list(dt.data.filter(regex='^len_')))
        out.append(dt.data)
    
    data = pd.concat(out, axis=0)
    data.to_csv(os.path.join('tabular_datasets', '{}_{}.tsv.gz'.format(rbp, analysis_name)), compression='gzip', sep="\t")
    
    return data


In [4]:
#CODE TO GENERATE DATASETS 
# kwargs = {'subset_RBPs': 'encode_in_rosina2017'}   

# data_config = {'occurrences_alone': [True, False, False],
#                'occurrences_based_on_location': [False, True, False],
#                'occurrences_based_on_location_and_distances': [False, False, True]}

# SPLICEAI_PREDS = SPLICEAI_PREDS[SPLICEAI_PREDS.RBP == "CELF1"]
# all_datasets = {}
# for analysis, flags in data_config.items():
#     df = SPLICEAI_PREDS.groupby(['RBP']).progress_apply(generateDatasets, 
#                                                         analysis_name=analysis,
#                                                         use_motif_counts=flags[0],
#                                                         use_motif_locations=flags[1],
#                                                         use_motif_distances=flags[2],
#                                                         **kwargs).reset_index(drop=True)
    
#     all_datasets[analysis] = df

In [5]:
# CODE TO LOAD PROCESSED DATASETS
all_datasets = {'occurrences_alone': pd.read_csv('tabular_datasets/CELF1_S1.tsv.gz', sep="\t"),
               'occurrences_based_on_location': pd.read_csv('tabular_datasets/CELF1_S2.tsv.gz', sep="\t")#,
               #'occurrences_based_on_location_and_distances': pd.read_csv('tabular_datasets/CELF1_S3.tsv.gz', sep="\t")
               }

In [6]:
for k, v in all_datasets.items():
    all_datasets[k] = v.drop([x for x in cols_to_drop if x in v.columns], axis=1)

In [234]:
from typing import Union
def plotFeatureImportances(df: pd.DataFrame, outdir: str, 
                           basename:str, 
                           correlated_pairs: dict = None, 
                           ranks_pval: Union[pd.Series, pd.DataFrame] = None):
 
    def _draw(df, rbp: str, basename:str, ranks_pval, is_correlated_feat: bool = False):
            
        clrs = ['maroon' if x is True else 'grey' for x in df.is_target]
        
        fig, ax = plt.subplots(figsize=(10, 9))
        g = sns.barplot(x='feat_import', y='feat_names_l' if 'feat_names_l' in df.columns else 'feat_names', orient='h', palette=clrs, data=df.head(40))
    
        try:
            idx = clrs.index('maroon')
            for i, p in enumerate(g.patches):

                if i == idx:
               
                    g.annotate('pval: {}'.format(str(ranks_pval[rbp][1]) if is_correlated_feat is False else str(ranks_pval[rbp][3])), 
                            xy=(p.get_x() + p.get_width() + 0.001, p.get_y()), 
                            ha = 'left', va = 'top')#,
                    #       textcoords = 'offset points')

        except ValueError:
            a=1
            
        if is_correlated_feat:
            out = '{}/{}_{}_feat_import_using_corrFeat.pdf'.format(outdir, rbp, basename)
        else:
            out = '{}/{}_{}_feat_import.pdf'.format(outdir, rbp, basename)
            
        plt.tight_layout()
        plt.savefig(out)
        plt.close()

    rbp = df.name
    df = df[df.feat_import > 0].sort_values('feat_import', ascending=False)
    
    if correlated_pairs is not None:
        
        corr_feat_at_rbp = correlated_pairs[rbp]
        df['feat_names_l'] = df.feat_names.apply(lambda x: x + " (n_corr={})".format(str(len(corr_feat_at_rbp[x]))))
    
    # True RBP-related values    
    _target_features = df[df.feat_names.str.contains(rbp)].feat_names  
    df['is_target'] = df.feat_names.isin(_target_features) 
    _draw(df, rbp, basename, ranks_pval)
    
    # Accounting for highly correlated features
    if all(x is not None for x in [correlated_pairs, ranks_pval]):
        
        corr_feat_at_rbp = correlated_pairs[rbp][rbp] + df[df.feat_names.str.contains(rbp)].feat_names.tolist()

        mask = df.feat_names.str.contains('|'.join(corr_feat_at_rbp))
        df['is_target'] = mask
  
        _draw(df, rbp, basename, ranks_pval, is_correlated_feat=True) 
        


In [235]:
import random
def do_simulations(feat_names, target_features, rank_target):
    n_permutations = 1000
    
    smaller_rank = 0
    for i in range(n_permutations):
    
        #shuffling inplace
        random.shuffle(feat_names)

        _target_features = feat_names[feat_names.isin(target_features)]  
        _target_features_rank = _target_features.index[0]
 
        if _target_features_rank < rank_target:
            smaller_rank += 1

    pval = smaller_rank / n_permutations       
    return pval

def testRanksOfImportances(df: pd.DataFrame, correlated_pairs: dict = None):
    rbp== df.name
    ranks = df.sort_values('feat_import', ascending=False).feat_names.reset_index(drop=True)
    
    target_features = ranks[ranks.str.contains(rbp)]

    if target_features.size > 0:
        # First rank is kept
        target_features_rank = target_features.index[0]
        
        # Median of ranks for the features belonging to a single RBP
        # target_features_rank = np.median(target_features.index)
        pval_raw = do_simulations(ranks, target_features.tolist(), target_features_rank)
    
    else:
        pval_raw, target_features_rank = None, None
        
    if correlated_pairs:
        corr_feat_at_rbp = correlated_pairs[rbp][rbp]
      
        target_features = ranks[ranks.str.contains('|'.join(corr_feat_at_rbp))]
        
        if target_features.size > 0:
            target_features_rank_using_corr_feat = target_features.index[0]
            pval_using_match_to_corr_feat = do_simulations(ranks, target_features.tolist(), target_features_rank_using_corr_feat)

    else:
        pval_using_match_to_corr_feat = None
        target_features_rank_using_corr_feat = None
    
    return target_features_rank, pval_raw, target_features_rank_using_corr_feat, pval_using_match_to_corr_feat


## Predict exon strength based on motif frequencies at different levels of resolution

In [12]:
from sklearn.dummy import DummyRegressor

def dummy_regressor(df: pd.DataFrame, value_to_predict: str = "average_cassette_strength"):
    """
    Dummy regressor to serve as baseline 
    to compare with other models
    """
    if df.shape[0] > 100:
        df = df.drop(['seq_id', 'target_coordinates', 'exon_group', 'RBP'], axis=1)
        
        y = df[value_to_predict]
        x = df.drop(value_to_predict, axis=1)
        
        dummy_regr = DummyRegressor(strategy="median")
        
        cv_results_r2 = cross_validate(dummy_regr, x, y, scoring='r2', cv=10, return_estimator=True)
        cv_results_mse = cross_validate(dummy_regr, x, y, scoring='neg_mean_squared_error', cv=10, return_estimator=True)

        mses, r2_scores = [],[]
        
        for i, model in enumerate(cv_results_mse['estimator']):
            
            #tree.plot_tree(rgr, feature_names=x.columns)
            mses.append(cv_results_mse['test_score'][i])
            r2_scores.append(cv_results_r2['test_score'][i])
        
        return round(np.mean([abs(x) for x in mses]), 3), round(np.mean(r2_scores), 3), None, None
    
    else:
        print("Not enough data: {}".format(df.name))
        return

In [13]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import tree

def dt_regressor(df: pd.DataFrame, 
                 value_to_predict: str = 'average_cassette_strength'):
    """
    Simple regressor to estimate exon strength 
    based on motif ocurrences features
    """
    
    if df.shape[0] > 100:
        df = df.drop(['seq_id', 'target_coordinates', 'exon_group', 'RBP'], axis=1)
        
        y = df[value_to_predict]
        x = df.drop(value_to_predict, axis=1)
        
        rgr = tree.DecisionTreeRegressor(random_state=0, min_samples_leaf=3, max_depth=5)
        

        cv_results_r2 = cross_validate(rgr, x, y, scoring='r2', cv=10, return_estimator=True)
        cv_results_mse = cross_validate(rgr, x, y, scoring='neg_mean_squared_error', cv=10, return_estimator=True)

        mses, r2_scores, feat_import = [],[],[]
        
        for i, model in enumerate(cv_results_mse['estimator']):
            
            #tree.plot_tree(rgr, feature_names=x.columns)
            feat_import.append(model.feature_importances_)
            mses.append(cv_results_mse['test_score'][i])
            r2_scores.append(cv_results_r2['test_score'][i])
        
        return round(np.mean([abs(x) for x in mses]), 3), round(np.mean(r2_scores), 3), np.array(feat_import).mean(axis=0).tolist(), list(x)
    
    else:
        print("Not enough data: {}".format(df.name))
        return

In [14]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

def rf_regressor(df: pd.DataFrame, value_to_predict: str = 'average_cassette_strength'):
    """
    Simple random forest classifier to distinguish exon group 
    based on motif ocurrences features
    """
    if df.shape[0] > 100:
        df = df.drop(['seq_id', 'target_coordinates', 'exon_group', 'RBP'], axis=1)
        
        y = df[value_to_predict]
        x = df.drop(value_to_predict, axis=1)
    
        rgr = RandomForestRegressor(max_depth=5, random_state=0)
        
        cv_results_r2 = cross_validate(rgr, x, y, scoring='r2', cv=5, return_estimator=True)
        cv_results_mse = cross_validate(rgr, x, y, scoring='neg_mean_squared_error', cv=5, return_estimator=True)
        
        mses, r2_scores, feat_import = [], [], []
        
        for i, model in enumerate(cv_results_mse['estimator']):
            
            #tree.plot_tree(rgr, feature_names=x.columns)
            feat_import.append(model.feature_importances_)
            mses.append(cv_results_mse['test_score'][i])
            r2_scores.append(cv_results_r2['test_score'][i])
                 
        return round(np.mean([abs(x) for x in mses]), 3), round(np.mean(r2_scores), 3), np.array(feat_import).mean(axis=0).tolist(), list(x)
    
    else:
        print("Not enough data: {}".format(df.name))
        return

In [15]:
import xgboost as xgb
from sklearn.preprocessing import LabelBinarizer

def xgb_regressor(df: pd.DataFrame, 
                  value_to_predict: str = 'average_cassette_strength'):
    """
    Simple xgboost classifier to distinguish exon group 
    based on motif ocurrences features
    """
    if df.shape[0] > 100:
        df = df.drop(['seq_id', 'target_coordinates', 'exon_group', 'RBP'], axis=1)
        
        y = df[value_to_predict]
        x = df.drop(value_to_predict, axis=1)
        #y = LabelBinarizer().fit_transform(y).ravel()
    
        params = {'max_depth': 5, 
                  'eta': 0.3,
                  'eval_metric': 'logloss'}

        rgr = xgb.XGBRegressor(use_label_encoder=False, **params)
        
        cv_results_r2 = cross_validate(rgr, x, y, scoring='r2', cv=5, return_estimator=True)
        cv_results_mse = cross_validate(rgr, x, y, scoring='neg_mean_squared_error', cv=5, return_estimator=True)
        
        mses, r2_scores, feat_import = [], [], []
        
        for i, model in enumerate(cv_results_mse['estimator']):
            
            #tree.plot_tree(rgr, feature_names=x.columns)
            feat_import.append(model.feature_importances_)
            mses.append(cv_results_mse['test_score'][i])
            r2_scores.append(cv_results_r2['test_score'][i])
                 
            
        return round(np.mean([abs(x) for x in mses]), 3), round(np.mean(r2_scores), 3), np.array(feat_import).mean(axis=0).tolist(), list(x)
    
    else:
        print("Not enough data: {}".format(df.name))
        return

In [ ]:
regressors = { 'Dummy_Regr': dummy_regressor,
               'DecisionTree_Regr': dt_regressor, 
               'RandomForest_Regr': rf_regressor,
               'XGBoost_Regr': xgb_regressor}

out_rgr = []
for dataset_type, df in all_datasets.items():
    print("Dataset type: {}".format(dataset_type))
    
    for rgr_name, rgr in regressors.items():
        print("Regressor: {}".format(rgr_name))
        output_rgr = df.groupby('RBP').progress_apply(rgr).reset_index().rename({0: 'out'}, axis=1).dropna()

        output_rgr[['mse', 'r2_score', 'feat_import', 'feat_names']] = pd.DataFrame(output_rgr.out.tolist(), index=output_rgr.index)
        output_rgr.drop('out', axis=1, inplace=True)
        output_rgr['regressor'] = rgr_name
        output_rgr['dataset'] = dataset_type
        output_rgr['RBP'] = output_rgr.RBP.apply(lambda x: x + " (N={})".format(str(df[df.RBP == x].shape[0])))
        out_rgr.append(output_rgr)
        
        out_path = os.path.join(dataset_type, rgr_name)
        os.makedirs(out_path, exist_ok=True)
        
        if "Dummy" not in rgr_name:
            feat_import_df = output_rgr.drop(['mse', 'r2_score'], axis=1).explode(['feat_import', 'feat_names'])
            feat_import_df.groupby('RBP').apply(plotFeatureImportances, outdir=out_path)     

out_rgr = pd.concat(out_rgr)
out_rgr.to_csv('regression_exonStrength.tsv.gz', compression='gzip', sep="\t")

In [17]:
%%capture
out_rgr['dataset'] = out_rgr.dataset.replace({'occurrences_alone': '_1',
                                                'occurrences_based_on_location': '_2',
                                                'occurrences_based_on_location_and_distances': '_3'})

out_rgr['regressor'] = out_rgr.regressor.str.replace('_Regr', '')
out_rgr['rgr'] = out_rgr.regressor + out_rgr.dataset

r2score_heat = out_rgr.drop(columns=['feat_import', 'feat_names', 'mse']).pivot(index='RBP', columns='rgr', values='r2_score')
g = sns.clustermap(data=r2score_heat, cmap="vlag", figsize=(12,12), yticklabels=True)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize=12)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize=7)
plt.savefig('regression_exonStrength_r2_scores.pdf')

mse_heat = out_rgr.drop(columns=['feat_import', 'feat_names', 'r2_score']).pivot(index='RBP', columns='rgr', values='mse')
g = sns.clustermap(data=mse_heat, cmap="vlag", figsize=(12,12), yticklabels=True)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize=12)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize=7)
plt.savefig('regression_exonStrength_mse_scores.pdf')

## Predict dPSI changes based on motif frequencies at different levels of resolution

### For that, we need the true dPSI changes observed in the splicing analysis

In [ ]:
dpsi_data_path = "/Users/pbarbosa/git_repos/interpret_splicing/notebooks/1_BUILD_DATASETS/7_final_datasets/ALL_CONCAT.tsv.gz"
dpsi_data = pd.read_csv(dpsi_data_path, sep="\t")

kd_data = dpsi_data[['exon_KD', 'Strand_KD', 'gene_name_KD', 'transcript_id_KD', 'transcript_type_KD', 'Existing_exon_KD', 'dPSI_KD', 'RBP']].copy()
kd_data.columns = [x.replace("_KD", "") for x in kd_data.columns]
kd_data['exon_group'] = "KD"

ctrl_data = dpsi_data[['exon_CTRL', 'Strand_CTRL', 'gene_name_CTRL', 'transcript_id_CTRL', 'transcript_type_CTRL', 'Existing_exon_CTRL', 'dPSI_CTRL', 'RBP']].copy()
ctrl_data.columns = [x.replace("_CTRL", "") for x in ctrl_data.columns]
ctrl_data['exon_group'] = 'CTRL'
dpsi_cols = ['target_coordinates', 'RBP', 'exon_group', 'dPSI']
dpsi_data = pd.concat([kd_data, ctrl_data]).rename({'exon': 'target_coordinates'}, axis=1)[dpsi_cols]

In [ ]:
all_datasets_with_dpsi = {}
for k, v in all_datasets.items():
    with_dpsi = pd.merge(v, dpsi_data, on=dpsi_cols[:-1]).drop('average_cassette_strength', axis=1)
    all_datasets_with_dpsi[k] = with_dpsi

In [ ]:
regressors = {'Dummy_Regr_dPSI': dummy_regressor,
              'DecisionTree_Regr_dPSI': dt_regressor, 
               'RandomForest_Regr_dPSI': rf_regressor,
               'XGBoost_Regr_dPSI': xgb_regressor
}
out_rgr = []
for dataset_type, df in all_datasets_with_dpsi.items():
    print("Dataset type: {}".format(dataset_type))
    
    for rgr_name, rgr in regressors.items():
        print("Regressor: {}".format(rgr_name))

        output_rgr = df.groupby('RBP').progress_apply(rgr, value_to_predict='dPSI').reset_index().rename({0: 'out'}, axis=1).dropna()

        output_rgr[['mse', 'r2_score', 'feat_import', 'feat_names']] = pd.DataFrame(output_rgr.out.tolist(), index=output_rgr.index)
        output_rgr.drop('out', axis=1, inplace=True)
        output_rgr['regressor'] = rgr_name
        output_rgr['dataset'] = dataset_type
        output_rgr['RBP'] = output_rgr.RBP.apply(lambda x: x + " (N={})".format(str(df[df.RBP == x].shape[0])))
        out_rgr.append(output_rgr)
        
        out_path = os.path.join(dataset_type, rgr_name)
        os.makedirs(out_path, exist_ok=True)
        
        if 'Dummy' not in rgr_name:
            feat_import_df = output_rgr.drop(['mse', 'r2_score'], axis=1).explode(['feat_import', 'feat_names'])
            feat_import_df.groupby('RBP').apply(plotFeatureImportances, outdir=out_path)     

out_rgr = pd.concat(out_rgr)
out_rgr.to_csv('regression_dPSI.tsv.gz', compression='gzip', sep='\t')

In [ ]:
%%capture
out_rgr['dataset'] = out_rgr.dataset.replace({'occurrences_alone': '_1',
                                              'occurrences_based_on_location': '_2',
                                              'occurrences_based_on_location_and_distances': '_3'})

out_rgr['regressor'] = out_rgr.regressor.str.replace('_Regr', '')
out_rgr['rgr'] = out_rgr.regressor + out_rgr.dataset

r2score_heat = out_rgr.drop(columns=['feat_import', 'feat_names', 'mse']).pivot(index='RBP', columns='rgr', values='r2_score')
g = sns.clustermap(data=r2score_heat, cmap="vlag", figsize=(12,12), yticklabels=True)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize=12)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize=7)
plt.savefig('regression_dPSI_r2_scores.pdf')


mse_heat = out_rgr.drop(columns=['feat_import', 'feat_names', 'r2_score']).pivot(index='RBP', columns='rgr', values='mse')
g = sns.clustermap(data=mse_heat, cmap="vlag", figsize=(12,12), yticklabels=True)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize=12)
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize=7)
plt.savefig('regression_dPSI_mse_scores.pdf')